In [ ]:
!nvidia-smi
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive'
os.chdir(path)
os.listdir(path)

In [72]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F


class Model(torch.nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.l1 = torch.nn.Linear(784,512)
    self.l2 = torch.nn.Linear(512,256)
    self.l3 = torch.nn.Linear(256,128)
    self.l4 = torch.nn.Linear(128,64)
    self.l5 = torch.nn.Linear(64,10)
  def forward(self, x):
    '''
    输入的数据行状为(n张图片*1*28*28)
    为让模型处理数据，将输入reshape为(n张图片*784)
    784=1*28*28
    -1则表示，reshape时自动计算第二个维度大小为784时，第一个维度的大小
    即，自动计算，已知特征数为784时，样本的数量
    '''
    x = x.view(-1,784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x) #使用CrossEntropyLoss时，神经网络最后一层不需要加激活函数

batch_size = 64
#Compose组合[]内的操作，顺序执行
transform = transforms.Compose([
  transforms.ToTensor(),          #把PIL Image(像素/维度28*28，每个像素取值0,1,2...,255)，转换成tensor(1*28*28 即channel*width*height，取值0~1)
  transforms.Normalize((0.1307,),(0.3081,)) #数据标准化，Normalize((均值,),(标准差,))                 
])

train_set = datasets.MNIST(root='Colab Notebooks/dataset',#获取MNIST数据集并储存在本地指定位置
              train=True,           # train : True = 训练集
              transform=transform,       #把数据转换成tensor
              download=True)
test_set = datasets.MNIST(root='Colab Notebooks/dataset',
              train=False,          # train : False = 测试集
              transform=transform,
              download=True)

#加载数据集
train_loader = DataLoader(dataset=train_set,
              batch_size=batch_size,
              shuffle=True)
test_loader = DataLoader(dataset=test_set,
              batch_size=batch_size,
              shuffle=False)#测试集用于测试，无必要做shuffle

model = Model()
criterion = torch.nn.CrossEntropyLoss()
#使用CrossEntropyLoss时，神经网络最后一层不需要加激活函数。且target标签需要为LongTensor
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
  running_loss = 0.0
  for batch_idx,(input,target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:#在每次的epoch中，每过300条数据输出一次
      print('[%d,%5d] loss: %.3f' % (epoch + 1, batch_idx +1, running_loss/300))
      running_loss = 0.0

def test():
  correct = 0
  total = 0
  with torch.no_grad(): #torch.no_grad()不计算梯度
    for input,target in test_loader:
      output = model(input)
      _,predicted = torch.max(output.data,dim=1)#求10个里最大值的小标，对比第2个维度里找最大值。行是第一个维度，列是第二个维度。取一行里最大列的下标
      total+=target.size(0)#求所有数据的条数
      correct += (predicted == target).sum().item()
  print('Accuracy on test set:%d %%' % (100*correct/total))


for epoch in range(10):
  train(epoch)
  test()




[1,  300] loss: 2.181
[1,  600] loss: 0.827
[1,  900] loss: 0.430
Accuracy on test set:89 %
[2,  300] loss: 0.328
[2,  600] loss: 0.280
[2,  900] loss: 0.231
Accuracy on test set:94 %
[3,  300] loss: 0.182
[3,  600] loss: 0.177
[3,  900] loss: 0.155
Accuracy on test set:95 %
[4,  300] loss: 0.123
[4,  600] loss: 0.137
[4,  900] loss: 0.107
Accuracy on test set:96 %
[5,  300] loss: 0.098
[5,  600] loss: 0.098
[5,  900] loss: 0.086
Accuracy on test set:96 %
[6,  300] loss: 0.076
[6,  600] loss: 0.076
[6,  900] loss: 0.072
Accuracy on test set:97 %
[7,  300] loss: 0.061
[7,  600] loss: 0.062
[7,  900] loss: 0.060
Accuracy on test set:97 %
[8,  300] loss: 0.050
[8,  600] loss: 0.050
[8,  900] loss: 0.051
Accuracy on test set:97 %
[9,  300] loss: 0.037
[9,  600] loss: 0.041
[9,  900] loss: 0.046
Accuracy on test set:97 %
[10,  300] loss: 0.030
[10,  600] loss: 0.032
[10,  900] loss: 0.037
Accuracy on test set:97 %
